tempo:
- ammazzare 

- ingannare 

- pioggia 

- mele 

- spazio

Cercare le parole nelle frasi del dataset, restiruire le frasi che le cotnengono.
pulire le frasi ottenute, per restituire "tempo" da "ammazzare il tempo" (stopwords e initialwords)
restituirer la parola più frequente tra quelle restituite

In [228]:
from nltk.corpus import stopwords
from translate import Translator
#print(stopwords.words('italian'))

In [229]:
WORDS_1 = ['ammazzare', 'ingannare', 'pioggia', 'mele', 'spazio']
WORDS_2 = ['bicchiere', 'mulino', 'gocce', 'gola', 'buco']
WORDS_3 = ['campa', 'battaglia', 'razza', 'Troia', 'vecchio']

In [230]:
def remove_stop_words_ita(phrase):
    stop_words = stopwords.words('italian')
    phrase = phrase.split()
    phrase = [word for word in phrase if word not in stop_words]
    return phrase

In [231]:
def check_dataset(path, words):
    phrases = []    
    with open(path, 'r') as file:
        for line in file:
            for word in words:
                if word.lower() in line.lower():  # Salviamo le frasi che contengono le parole in input
                    line = line.lower()
                    line = line.replace(word, '')
                    phrases.append(remove_stop_words_ita(line)) # Rimuoviamo dalla frase le stopword e le parole in input
    return phrases

In [232]:
def most_frequent_words(phrases):
    words = {}
    for phrase in phrases:
        for word in phrase:
            if word in words:
                words[word] += 1
            else:
                words[word] = 1
    return words

def get_most_frequent_word(words):
    max_word = ''
    max_count = 0
    for word in words:
        if words[word] > max_count:
            max_word = word
            max_count = words[word]
    return max_word

In [233]:
ph = check_dataset('..\ghigliottina.txt', WORDS_3)
print(ph)
dict = most_frequent_words(ph)
print(dict)
answer = get_most_frequent_word(dict)
print(f"\nLa sesta parola è: {answer}")

[['cavallo'], ['stare', 'na'], ['cavallo'], ['cavallo'], ['cavallo', 'troia'], ['grande', 'fratellogrande'], ['povero', 'quantunque', 'sentisse', 'bene', 'rischioso', 'gioco', 'giocava', 'paura', 'portare', 'soccorso', 'pisa', 'pure', 'volle', 'mancare:', '[â€¦]'], ['sordo', 'na'], ['stare', 'na'], ['tirare', 're'], ['vecchia', 'guardia', 'cucco', 'cucã¹', 'bacucco', 'primo', 'topo', 'toscana'], ['mondo'], ['(o', 'aspetta)', 'cavallo', 'erba', 'cresce'], ['cavallo', 'tienilo', 'orzo', 'paglia'], ['cavallo', 'giovane', 'porta', 'soldati', 'cavallo', 'porta', 'concime'], ['cavallo', 'tardi', 'muta', 'ambiatura'], ['va', 'cavallo', 'giovane', 'va', 'piedi'], ['giovine', 'mano', 'uccello', 'mano', 'ragazzo', 'cavallo', 'mano', 'frate', 'son', 'tre', 'cose', 'strapazzate']]
{'cavallo': 11, 'stare': 2, 'na': 3, 'troia': 1, 'grande': 1, 'fratellogrande': 1, 'povero': 1, 'quantunque': 1, 'sentisse': 1, 'bene': 1, 'rischioso': 1, 'gioco': 1, 'giocava': 1, 'paura': 1, 'portare': 1, 'soccorso': 1

### Codice per traduzione con google (pip install translate)
L'idea è di tradurre i concetti in inglese, e poi cercare relazioni su wordnet. oppure usare babelneet, ma forse serve una chiave per usarlo

IDEA SENZA SENSO, perchè se traduci tempo ottieni time, e per noi tempo vuol dire anche tempo atmosferico, in inglese invece no.

In [234]:
translator = Translator(to_lang='it')
translation = translator.translate('god')
print(translation)

Dio
